(info about pisces dataset) 
Here using the cif files from pisces dataset we are extracting the angles (phi and psi omega ?) and the sequences. The sequences are also have been put in the embedding space using Prot-Bert model for the ease of use. All angles are below 129 residues. 

## Importing libraries

In [1]:
#accessing files
import os

#manipulation
import numpy as np
import torch
import pandas as pd

#for python object structure
import pickle

#biopython for parsing the cif files and manipulation of the cif files
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB import PDBIO
from Bio import PDB

#data structures for manipulation
from typing import Dict, Tuple

#transformer model for embedding space creation
from transformers import BertModel, BertTokenizer

/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Changing to GPU

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Getting the sequences with length less than 128

In [3]:
file_path = 'cullpdb_pc30_0_res0_0_2_0_noBrks_noDsdr_len40_10000_R0_25_Nmr_d2024'
df = pd.read_csv(file_path, delim_whitespace=True)

#pdb_ids = df['PDBchain'].tolist()

df = df[df['len'] <= 128]
pdb_ids = df['PDBchain'].tolist()
print(pdb_ids)

#we need the chain acces information
pdb_id_map = {pdb_id[:-1]: pdb_id for pdb_id in pdb_ids}
print(pdb_id_map)

mapping_file = os.path.join( 'new_training/pdb_id_mapping.csv')
#pd.Series(pdb_id_map).to_csv(mapping_file, header=False)
pd.Series({k.lower(): v for k, v in pdb_id_map.items()}).to_csv(mapping_file, header=False)

['1E0ZA', '1RY4A', '1UG7A', '1WHNA', '2AYAA', '2D87A', '2DB5A', '2DKPA', '2EBKA', '2ETTA', '2FM4A', '2I4KA', '2JQ5A', '2K0RA', '2K5TA', '2LR4A', '2LS01', '2LVLA', '2MAHA', '2NCNA', '2RLOA', '2RV8A', '2YTUA', '4AR0A', '5AIWA', '5H7UA', '5N9QA', '8E22X', '1CMOA', '1DDFA', '1EALA', '1PQNA', '1QKLA', '1UEYA', '1W2QA', '1WH4A', '1WJRA', '1X5MA', '1ZU1A', '2CSOA', '2DMKA', '2EE7A', '2L1LB', '2LOEA', '2MF7A', '2MPLA', '2NBOA', '2TMPA', '4UEIA', '5M1UA', '7XRWA', '8OILA', '1RSFA', '1V5SA', '1X5EA', '1ZO0A', '2A4HA', '2CR4A', '2DAVA', '2IF1A', '2JP2A', '2JVNA', '2K2VA', '2L3GA', '2L57A', '2LBCA', '2LLNA', '2M26A', '3MSPA', '7F7NA', '7KNVA', '8SQXA', '1BUQA', '1BW3A', '1FR0A', '1GIOA', '1MH6A', '1PSYA', '1UENA', '1WYJA', '2AI6A', '2DHJA', '2K4VA', '2K4ZA', '2KA5A', '2KFPA', '2KYTA', '2L2CA', '2LDUA', '2LM0A', '2MDKA', '2MV4A', '2N2HB', '2NBGA', '2PRFA', '4A53A', '5KNWA', '5T3YA', '5WOEA', '6GWMA', '6TH8A', '6UF2A', '6XEHA', '7BQCA', '7PRDA', '7QYIA', '1DC7A', '1E8EA', '1EO1A', '1PIRA', '1T3VA', 

/tmp/ipykernel_234196/3753862082.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True)


In [6]:

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False ) # change model and tokenizer to t5
model_embedd = BertModel.from_pretrained("Rostlab/prot_bert")
model_embedd = model_embedd.to(device)


## Dividing data into batches

In [5]:

def extract_input(pdb_file, model, tokenizer):
    '''
    Input =>
    pdb_file : file containing all the information for the given protein (.pdb or .cif)
    tokenizer : tokenizer for the sequence tokenization
    model : transformer model used for creating embedding space
    
    Output =>
    sequence : extracted from the file
    angle_tensor : tensor containing phi and psi angles for the given sequence
    encoded_input : encoded sequence using the tokenizer
    embeddings : embedding created using the model for the given sequence
    '''
    #Parsing the cif file
    cif_parser = MMCIFParser()
    structure = cif_parser.get_structure("protein", pdb_file) # getting structure
    model0 = structure[0]

    #This is to access the dictionary keys, which are the same as the file name, 
    #Once the keys are accessed we can find which chain to take
    mapping_file = os.path.join('new_training/pdb_id_mapping.csv') 
    
    pdb_id_map = pd.read_csv(mapping_file, header=None, index_col=0).squeeze().to_dict()

    #extracting the file name as it contains the id
    filename = os.path.basename(pdb_file)
    pdb_id = filename[:4] # the first 4 letters of the file name is the dictionary key,whose value is the chain to be used

    full_pdb_id = pdb_id_map.get(pdb_id, None) #pdb id with the correct chain

    chain_A = model0[full_pdb_id[-1]]  # and we get chain A , the last letter of the id is the chain
    print("full pd ib:", full_pdb_id)

    #This is a very common need in bioinformatics of proteins
    d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
    'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
    'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
    'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

    sequence = []
    for residue in chain_A:
        #For simplicity we can use X for heteroatoms (ions and water)
        sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
    

    structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

    #Creating a dictionary to collect chain ids and turn them into indices
    chain_id_to_num = {}
    for num, chain in enumerate(model0.get_chains()):
        chain_id_to_num[chain.id] = num

    #Iterator of chains, turns it into list, [0] first chain
    chain:PDB.Chain.Chain = list(model0.get_chains())[chain_id_to_num.get(full_pdb_id[-1])]
    #This accesses the internal chain coords of the chain object
    ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord 

    d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

    phi_angles_list = []
    psi_angles_list = []

    for key in d:
      if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
          phi_angles_list.append(d[key].angle)
      elif key[0].akl[3] == 'C' and key[1].akl[3] == 'N' and key[2].akl[3] == 'CA' and key[3].akl[3] == 'C':
        psi_angles_list.append(d[key].angle)

    structure.internal_to_atom_coordinates(verbose = False)
    io = PDBIO() #this is to write a pdb/cif file again
    io.set_structure(structure)#setting the structure to the desired strcuture in the given file
    phi_angles_list.append(0)
    psi_angles_list.append(0)

    phi = np.asarray(phi_angles_list,dtype=np.float32)
    psi = np.asarray(psi_angles_list,dtype=np.float32)
    angles = np.vstack((psi,phi))
    angle_tensor = torch.tensor(angles, dtype=torch.float32).to(device)

    #Encoding the sequence
    encoded_input = tokenizer.encode(sequence, padding=, return_tensors='pt').to(device)  

    #Create the embedding using the model
    with torch.no_grad():
        outputs = model(input_ids=encoded_input)

    embeddings = outputs.last_hidden_state

    return sequence, angle_tensor, encoded_input, embeddings 

In [7]:
len(df) # number of proteins we need to extract in PISCES data

1723

You need to upload a huge folder of all the data here. The pdb files folder in directory path. In target path upload the inputs folder

In [8]:
#Data path
directory_path = 'small_proteins'
#Processed data path
target_path = 'new_training/'

embeddings_file = os.path.join(target_path, 'embeddings.pt')
angles_file = os.path.join(target_path, 'angles.pt')
sequences_file = os.path.join(target_path, 'sequences.pkl') 

#Embeddings tensor and angles tensor initialized with zero tensors
#The calculated values are concatenated, this way the padding is achieved to have equal length for each sequence
#128(max length) + 1 token start + 1 token end => 131 (python length starting from 0)
embeddings = torch.zeros(0, 131, 1024).to(device) 
angles = torch.zeros(0, 2, 131).to(device)
sequences = []

def pad_sequence(sequence, target_length=129, pad_char="X"): # may not need this as im using a padding in the tokenizer itself
    padding_length = target_length - len(sequence)
    padding = [pad_char] * padding_length
    return sequence + padding

#Get list of files in the directory containing pdb/cif files
file_list = os.listdir(directory_path)

for i,filename in enumerate(file_list):
    file_path = os.path.join(directory_path, filename)
   
    if os.path.isfile(file_path):
        sequence, angle, inputs, outputs, embedding, pdb_id= extract_input(file_path, model_embedd, tokenizer) # the key part that extracts all the information into a file
        if sequence is None or embedding is None or angle is None:
            print(f"Skipping {file_path} due to missing data")
            continue
        print(len(sequence))
        
        padded_sequence = pad_sequence(sequence)  # may not need this as im using a padding in the tokenizer itself
        sequences.append(padded_sequence)

        # Append the new data to the existing tensors
        embedding_length = min(embedding.size(1), 131)
        #print(np.shape(embedding))
        if i >= embeddings.size(0):
            embeddings = torch.cat([embeddings.to(device), torch.zeros(1, 131, 1024).to(device)], dim=0)
        embeddings[i, :embedding_length] = embedding[0, :, :]
        #print(angle.size)
        angle_length = min(angle.size(1), 131)
        if i >= angles.size(0):

            angles = torch.cat([angles.to(device), torch.zeros(1, 2, 131).to(device)], dim=0)
        
        angles[i, :, :angle_length] = angle[:, :]

        i += 1
        print(i)


# Save the updated tensors back to the files
torch.save(embeddings, embeddings_file)
torch.save(angles, angles_file)

with open(sequences_file, 'wb') as f: # you may not need to save the sequences
    pickle.dump(sequences, f)



start index:  0
['1e0z.cif', '1ry4.cif', '1ug7.cif', '1whn.cif', '2aya.cif', '2d87.cif', '2db5.cif', '2dkp.cif', '2ebk.cif', '2ett.cif', '2fm4.cif', '2i4k.cif', '2jq5.cif', '2k0r.cif', '2k5t.cif', '2lr4.cif', '2ls0.cif', '2lvl.cif', '2mah.cif', '2ncn.cif', '2rlo.cif', '2rv8.cif', '2ytu.cif', '4ar0.cif', '5aiw.cif', '5h7u.cif', '5n9q.cif', '8e22.cif', '1cmo.cif', '1ddf.cif', '1eal.cif', '1pqn.cif', '1qkl.cif', '1uey.cif', '1w2q.cif', '1wh4.cif', '1wjr.cif', '1x5m.cif', '1zu1.cif', '2cso.cif', '2dmk.cif', '2ee7.cif', '2l1l.cif', '2loe.cif', '2mf7.cif', '2mpl.cif', '2nbo.cif', '2tmp.cif', '4uei.cif', '5m1u.cif', '7xrw.cif', '8oil.cif', '1rsf.cif', '1v5s.cif', '1x5e.cif', '1zo0.cif', '2a4h.cif', '2cr4.cif', '2dav.cif', '2if1.cif', '2jp2.cif', '2jvn.cif', '2k2v.cif', '2l3g.cif', '2l57.cif', '2lbc.cif', '2lln.cif', '2m26.cif', '3msp.cif', '7f7n.cif', '7knv.cif', '8sqx.cif', '1buq.cif', '1bw3.cif', '1fr0.cif', '1gio.cif', '1mh6.cif', '1psy.cif', '1uen.cif', '1wyj.cif', '2ai6.cif', '2dhj.cif',

/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3746.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3802.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 7604.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 7660.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11462.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/

full pd ib: 1BUQA
letter:  0
126
73
73
full pd ib: 1BW3A
letter:  0
125
74
74
full pd ib: 1FR0A
letter:  0
125
75
75
full pd ib: 1GIOA
letter:  0
125
76
76
full pd ib: 1MH6A
letter:  0
125
77
77
full pd ib: 1PSYA
letter:  0
125
78
78
full pd ib: 1UENA
letter:  0
125
79
79
full pd ib: 1WYJA
letter:  0
125
80
80
full pd ib: 2AI6A
letter:  0
125
81
81
full pd ib: 2DHJA
letter:  0
125
82
82
full pd ib: 2K4VA
letter:  0
125
83
83
full pd ib: 2K4ZA
letter:  0
125
84
84
full pd ib: 2KA5A
letter:  0
125
85
85
full pd ib: 2KFPA
letter:  0
125
86
86
full pd ib: 2KYTA
letter:  0
125
87
87
full pd ib: 2L2CA
letter:  0
125
88
88
full pd ib: 2LDUA
letter:  0
125
89
89
full pd ib: 2LM0A
letter:  0
125
90
90
full pd ib: 2MDKA
letter:  0
125
91
91
full pd ib: 2MV4A
letter:  0
125
92
92
full pd ib: 2N2HB
letter:  1
125
93
93
full pd ib: 2NBGA
letter:  0
125
94
94
full pd ib: 2PRFA
letter:  0
125
95
95
full pd ib: 4A53A
letter:  0
125
96
96
full pd ib: 5KNWA
letter:  0
125
97
97
full pd ib: 5T3YA
letter:

/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 17056.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 17061.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 17066.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 17071.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 34128.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/

full pd ib: 2P80D
letter:  3
124
152
152
full pd ib: 2YQGA
letter:  0
123
153
153
full pd ib: 6NBNA
letter:  0
124
154
154
full pd ib: 8I25A
letter:  0
123
155
155
full pd ib: 1AXJA
letter:  0
123
156
156
full pd ib: 1BIPA
letter:  0
122
157
157
full pd ib: 1D4BA
letter:  0
122
158
158
full pd ib: 1DROA
letter:  0
122
159
159
full pd ib: 1IT4A
letter:  0
123
160
160
full pd ib: 1NR3A
letter:  0
122
161
161
full pd ib: 1PLOA
letter:  0
122
162
162
full pd ib: 1V5QA
letter:  0
122
163
163
full pd ib: 1YUAA
letter:  0
122
164
164
full pd ib: 2CQMA
letter:  0
122
165
165
full pd ib: 2CRYA
letter:  0
122
166
166
full pd ib: 2EQYA
letter:  0
122
167
167
full pd ib: 2KUTA
letter:  0
122
168
168
full pd ib: 2LFIA
letter:  0
122
169
169
full pd ib: 2LHTA
letter:  0
122
170
170
full pd ib: 2M5YA
letter:  0
122
171
171
full pd ib: 2MWIA
letter:  0
122
172
172
full pd ib: 2MZNA
letter:  0
122
173
173
full pd ib: 2ROHA
letter:  0
122
174
174
full pd ib: 2YS4A
letter:  0
122
175
175
full pd ib: 5Y7L

...

full pd ib: 2KA6B
letter:  1
45
1656
1656
full pd ib: 5X9XB
letter:  1
45
1657
1657


/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3550.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 7152.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10754.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 14356.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 17958.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bi

full pd ib: 1CF4B
letter:  1
44
1658
1658
full pd ib: 1CWXA
letter:  0
44
1659
1659
full pd ib: 1JUNA
letter:  0
44
1660
1660
full pd ib: 2K9DA
letter:  0
44
1661
1661
full pd ib: 2LB7A
letter:  0
44
1662
1662
full pd ib: 5KGZA
letter:  0
44
1663
1663
full pd ib: 6T33A
letter:  0
44
1664
1664
full pd ib: 7MJ3A
letter:  0
44
1665
1665
full pd ib: 1AW6A
letter:  0
45
1666
1666
full pd ib: 1BZKA
letter:  0
43
1667
1667


/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3841.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 7684.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11527.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 15370.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 19213.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bi

full pd ib: 2E30B
letter:  1
43
1668
1668
full pd ib: 2KE3A
letter:  0
43
1669
1669
full pd ib: 2KUYA
letter:  0
45
1670
1670
full pd ib: 2L2LA
letter:  0
43
1671
1671
full pd ib: 2LZLA
letter:  0
43
1672
1672
full pd ib: 2N70A
letter:  0
43
1673
1673
full pd ib: 5LMEA
letter:  0
45
1674
1674
full pd ib: 6OBKA
letter:  0
43
1675
1675
full pd ib: 8IMHA
letter:  0
43
1676
1676
full pd ib: 1CO4A
letter:  0
43
1677
1677
full pd ib: 1D6BA
letter:  0
42
1678
1678
full pd ib: 1GO9A
letter:  0
42
1679
1679
full pd ib: 1KV4A
letter:  0
42
1680
1680
full pd ib: 1QDPA
letter:  0
42
1681
1681
full pd ib: 1UJLA
letter:  0
42
1682
1682
full pd ib: 1WQKA
letter:  0
42
1683
1683
full pd ib: 2B4NA
letter:  0
42
1684
1684
full pd ib: 2LMZA
letter:  0
42
1685
1685
full pd ib: 8BVCA
letter:  0
42
1686
1686
full pd ib: 8UM1A
letter:  0
42
1687
1687
full pd ib: 1BI6H
letter:  7
41
1688
1688
full pd ib: 1HJ0A
letter:  0
41
1689
1689
full pd ib: 1WT7A
letter:  0
41
1690
1690
full pd ib: 2J5HA
letter:  0
41
16

In [9]:
print(np.shape(embedding[:,:,:]))
print(np.shape(embeddings[:,:,:]))

print(np.shape(angle))
print(np.shape(angles))

torch.Size([1, 42, 1024])
torch.Size([1711, 131, 1024])
torch.Size([2, 40])
torch.Size([1711, 2, 131])


In [10]:
angles = torch.load('new_training/angles.pt')
phi_angles = angles[:,0,:]
psi_angles = angles[:,1,:]

print(angles.size())
print(phi_angles.size())
print(angles[15,:,:])

torch.Size([1711, 2, 131])
torch.Size([1711, 131])
tensor([[-129.0686,  -93.1709, -107.1592,  -47.9783,  -67.9515,  -54.4657,
         -115.7745, -156.6232, -133.5739, -163.0471,  -53.1599,   62.5310,
         -138.5958, -149.2970, -156.4849, -136.3228,  -56.4204,  -74.6726,
         -109.2731,   64.7929,   56.2258,  -65.5943,  -61.9461,   42.9776,
          -50.0812,  -71.9293,  -55.8442,  -66.1385, -145.5454,  -48.5025,
         -121.4797, -143.9788,  -66.8832,  -63.2468, -154.5751,  -56.8206,
          -74.4059,  -69.6196,   94.3943, -131.0393,  -76.3750,  -82.8213,
         -115.5403,  -92.0608, -128.3188, -103.1864,  -90.8789, -113.1141,
          -87.8241,  114.8718,  -74.2119, -152.0556, -133.8653, -121.1926,
          -85.7341, -138.8487, -142.8164,  -81.4579,   59.6251,  -68.3115,
          -71.6146,   86.9223, -138.8716,  -66.5406,  -58.2832,  -68.3479,
         -121.7699,  -95.3596,  173.9619,  -89.7241,  -65.4513, -140.8306,
          -72.8081, -138.7966, -157.7205, -150.27